In [5]:
import json
import csv
import pandas as pd
from urllib.parse import parse_qs, urlparse, unquote
from typing import Dict, List, Tuple

class LogQAValidator:
    def __init__(self):
        self.scenarios = []
        self.logs = []
        
    def load_scenario_csv(self, csv_file_path: str):
        """CSV 파일에서 시나리오 로드"""
        self.scenarios = []
        df = pd.read_csv(csv_file_path, encoding='utf-8')
        
        # 컬럼 구성에 따라 고유키 조합 결정
        columns = df.columns.tolist()
        if 'page_id' in columns:
            self.unique_keys = ['page_id', 'click_type', 'act_type']
            print("🔑 고유키 조합: page_id + click_type + act_type")
        elif 'click_text' in columns:
            self.unique_keys = ['click_text', 'click_type', 'act_type']
            print("🔑 고유키 조합: click_text + click_type + act_type")
        else:
            raise ValueError("지원하지 않는 시나리오 형식입니다. page_id 또는 click_text 컬럼이 필요합니다.")
        
        for _, row in df.iterrows():
            key_list = [k.strip() for k in str(row['key_list']).split(',') if k.strip()]
            scenario = {
                'expected_keys': key_list
            }
            
            # 고유키 값들 추가
            for key in self.unique_keys:
                scenario[key] = str(row[key]) if pd.notna(row[key]) else ''
            
            self.scenarios.append(scenario)
        
        print(f"✅ 시나리오 로드: {len(self.scenarios)}개")

    def parse_logs_from_file(self, log_file_path: str):
        """로그 txt 파일에서 직접 파싱"""
        try:
            with open(log_file_path, 'r', encoding='utf-8') as file:
                log_content = file.read()
            self.parse_logs(log_content)
            print(f"✅ 로그 파일 로드: {log_file_path}")
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {log_file_path}")
        except Exception as e:
            print(f"❌ 파일 읽기 오류: {e}")

    def parse_logs(self, log_content: str):
        """로그 파싱"""
        self.logs = []
        for i, line in enumerate(log_content.strip().split('\n')):
            if not line.strip():
                continue
                
            try:
                log_data = json.loads(line.strip())
                if 'REQUEST' not in log_data:
                    continue
                    
                params = self._parse_url(log_data['REQUEST'])
                if params:
                    self.logs.append({
                        'line': i + 1,
                        'timestamp': log_data.get('LOGTIME', ''),
                        'params': params
                    })
            except json.JSONDecodeError:
                continue
                
        print(f"✅ 로그 파싱: {len(self.logs)}개")

    def _parse_url(self, request_url: str) -> Dict[str, str]:
        """URL 파라미터 파싱"""
        try:
            url = request_url.encode('utf-8').decode('unicode_escape')
            parsed = urlparse(url)
            query_params = parse_qs(parsed.query, keep_blank_values=True)
            
            result = {}
            for key, values in query_params.items():
                if values:
                    decoded = unquote(values[0], encoding='utf-8')
                    # 한글 깨짐 복구 시도
                    try:
                        if any(ord(c) > 127 for c in decoded):
                            decoded = decoded.encode('latin-1').decode('utf-8')
                    except:
                        pass
                    result[key] = decoded
            return result
        except:
            return {}

    def validate_and_export(self, output_file: str):
        """검증 수행 및 결과 XLSX 출력"""
        results = []
        
        for scenario in self.scenarios:
            # 시나리오와 매칭되는 로그 찾기
            matched_logs = self._find_matching_logs(scenario)
            
            if not matched_logs:
                # 매칭되는 로그가 없는 경우
                for key in scenario['expected_keys']:
                    result = {
                        'key': key,
                        'value': 'NOT_FOUND',
                        'pass': 'FAIL'
                    }
                    
                    # 고유키 값들 추가
                    for uk in self.unique_keys:
                        result[uk] = scenario.get(uk, '')
                    
                    results.append(result)
                continue
            
            # 각 매칭된 로그에 대해 검증
            for log in matched_logs:
                actual_keys = set(log['params'].keys())
                expected_keys = set(scenario['expected_keys'])
                
                # 예상 키들 검증
                for key in expected_keys:
                    value = log['params'].get(key, 'MISSING')
                    pass_status = 'PASS' if key in actual_keys else 'FAIL'
                    
                    result = {
                        'key': key,
                        'value': value,
                        'pass': pass_status
                    }
                    
                    # 고유키 값들 추가
                    for uk in self.unique_keys:
                        result[uk] = scenario.get(uk, '')
                    
                    results.append(result)
                
                # 예상치 못한 키들 (선택사항)
                unexpected_keys = actual_keys - expected_keys
                for key in unexpected_keys:
                    result = {
                        'key': key,
                        'value': log['params'][key],
                        'pass': 'UNEXPECTED'
                    }
                    
                    # 고유키 값들 추가
                    for uk in self.unique_keys:
                        result[uk] = scenario.get(uk, '')
                    
                    results.append(result)

        # XLSX 출력 - 컬럼 순서 정렬
        df_results = pd.DataFrame(results)
        
        # 고유키 조합에 따른 컬럼 순서 정의
        if 'page_id' in self.unique_keys:
            column_order = ['page_id', 'click_type', 'act_type', 'key', 'value', 'pass']
        else:  # click_text 기반
            column_order = ['click_text', 'click_type', 'act_type', 'key', 'value', 'pass']
        
        # 컬럼 순서 적용
        df_results = df_results[column_order]
        df_results.to_excel(output_file, index=False, engine='openpyxl')
        
        # 요약 출력
        self._print_summary(results)
        print(f"✅ 결과 저장: {output_file}")

    def _find_matching_logs(self, scenario: Dict) -> List[Dict]:
        """시나리오와 매칭되는 로그 찾기"""
        matched = []
        
        for log in self.logs:
            params = log['params']
            
            # 동적 고유키로 매칭
            is_match = True
            for key in self.unique_keys:
                if params.get(key, '') != scenario.get(key, ''):
                    is_match = False
                    break
            
            if is_match:
                matched.append(log)
        
        return matched

    def _print_summary(self, results: List[Dict]):
        """결과 요약 출력"""
        df = pd.DataFrame(results)
        
        total = len(df)
        passed = len(df[df['pass'] == 'PASS'])
        failed = len(df[df['pass'] == 'FAIL'])
        unexpected = len(df[df['pass'] == 'UNEXPECTED'])
        
        print(f"\n📊 검증 결과:")
        print(f"   전체: {total}개")
        print(f"   통과: {passed}개")
        print(f"   실패: {failed}개")
        print(f"   예상외: {unexpected}개")
        print(f"   성공률: {passed/total*100:.1f}%" if total > 0 else "   성공률: 0%")

# 사용 예시
def create_sample_scenario_csv():
    """샘플 시나리오 CSV 생성"""
    # Case 1: page_id 기반 시나리오
    sample_data_case1 = [
        {
            'page_id': 'life-dev/main',
            'click_type': '',
            'act_type': 'impression',
            'key_list': 'channel, page_url, page_id, act_type, banner_text'
        },
        {
            'page_id': 'life-dev/main',
            'click_type': '',
            'act_type': 'pageview',
            'key_list': 'channel, page_url, page_id, act_type'
        },
        {
            'page_id': 'life-dev/main',
            'click_type': '퀵버튼',
            'act_type': 'click',
            'key_list': 'channel, page_url, page_id, act_type, click_text, click_type'
        }
    ]
    
    # Case 2: click_text 기반 시나리오
    sample_data_case2 = [
        {
            'click_text': '로그인 버튼',
            'click_type': '버튼',
            'act_type': 'click',
            'key_list': 'channel, click_text, click_type, act_type, user_id'
        },
        {
            'click_text': '상품 이미지',
            'click_type': '이미지',
            'act_type': 'click',
            'key_list': 'channel, click_text, click_type, act_type, product_id'
        }
    ]
    
    # Case 1 파일 생성
    df1 = pd.DataFrame(sample_data_case1)
    df1.to_csv('scenario_case1.csv', index=False, encoding='utf-8')
    
    # Case 2 파일 생성
    df2 = pd.DataFrame(sample_data_case2)
    df2.to_csv('scenario_case2.csv', index=False, encoding='utf-8')
    
    print("✅ 샘플 시나리오 CSV 생성:")
    print("   - scenario_case1.csv (page_id 기반)")
    print("   - scenario_case2.csv (click_text 기반)")

def run_example():
    """실행 예시"""
    # 샘플 시나리오 CSV 생성
    create_sample_scenario_csv()
    
    # 검증기 초기화
    validator = LogQAValidator()
    
    # Case 1: page_id 기반 시나리오 테스트
    print("\n=== Case 1: page_id 기반 시나리오 테스트 ===")
    validator.load_scenario_csv('scenario_case1.csv')
    
    # 로그 파일에서 직접 파싱
    # validator.parse_logs_from_file('log_file.txt')
    
    # 또는 문자열로 파싱 (예시용)
    log_data = '''
{"REQUEST":"https://life-dev.hectoinnovation.co.kr/main?channel\\u003dRround\\u0026page_url\\u003dhttps://life-dev.hectoinnovation.co.kr/main\\u0026page_id\\u003dlife-dev/main\\u0026act_type\\u003dimpression\\u0026banner_text\\u003d애국가"}
{"REQUEST":"https://life-dev.hectoinnovation.co.kr/main?channel\\u003dRround\\u0026page_url\\u003dhttps://life-dev.hectoinnovation.co.kr/main\\u0026page_id\\u003dlife-dev/main\\u0026act_type\\u003dpageview"}
{"REQUEST":"https://life-dev.hectoinnovation.co.kr/main?channel\\u003dRround\\u0026page_url\\u003dhttps://life-dev.hectoinnovation.co.kr/main\\u0026page_id\\u003dlife-dev/main\\u0026act_type\\u003dclick\\u0026click_text\\u003d라운드로또\\u0026click_type\\u003d퀵버튼"}
'''
    validator.parse_logs(log_data)
    
    # 검증 및 결과 출력
    validator.validate_and_export('./result/validation_result_case1.xlsx')
    
    # Case 2: click_text 기반 시나리오 테스트
    print("\n=== Case 2: click_text 기반 시나리오 테스트 ===")
    validator2 = LogQAValidator()
    validator2.load_scenario_csv('scenario_case2.csv')
    
    log_data2 = '''
{"REQUEST":"https://example.com/login?channel\\u003dWeb\\u0026click_text\\u003d로그인 버튼\\u0026click_type\\u003d버튼\\u0026act_type\\u003dclick\\u0026user_id\\u003d123"}
{"REQUEST":"https://example.com/product?channel\\u003dWeb\\u0026click_text\\u003d상품 이미지\\u0026click_type\\u003d이미지\\u0026act_type\\u003dclick\\u0026product_id\\u003dABC123"}
'''
    validator2.parse_logs(log_data2)
    validator2.validate_and_export('./result/validation_result_case2.xlsx')

if __name__ == "__main__":
    run_example()

✅ 샘플 시나리오 CSV 생성:
   - scenario_case1.csv (page_id 기반)
   - scenario_case2.csv (click_text 기반)

=== Case 1: page_id 기반 시나리오 테스트 ===
🔑 고유키 조합: page_id + click_type + act_type
✅ 시나리오 로드: 3개
✅ 로그 파싱: 3개

📊 검증 결과:
   전체: 15개
   통과: 15개
   실패: 0개
   예상외: 0개
   성공률: 100.0%
✅ 결과 저장: ./result/validation_result_case1.xlsx

=== Case 2: click_text 기반 시나리오 테스트 ===
🔑 고유키 조합: click_text + click_type + act_type
✅ 시나리오 로드: 2개
✅ 로그 파싱: 2개

📊 검증 결과:
   전체: 10개
   통과: 10개
   실패: 0개
   예상외: 0개
   성공률: 100.0%
✅ 결과 저장: ./result/validation_result_case2.xlsx
